In [1]:
import json
import random
import textwrap
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bellem.utils import jprint

In [2]:
# df = pd.read_json('../data/generated/reports/results.jsonl', lines=True)
df = pd.read_json('results.jsonl', lines=True)
print(len(df))
df.head()

300


,id,n_hops,answer,hops,predicted_answer,reference_answers,exact_match,f1,fuzzy_match
0,2hop__592067_529125,2,,[{'question': 'What is the seat of the county ...,,[San Andreas],0,0,0
1,2hop__91929_86706,2,,"[{'question': 'In the US Open, who beat the pe...",,[Novak Djokovic],0,0,0
2,2hop__507792_754472,2,,[{'question': 'Where did the rapper of Midnigh...,,"[San Francisco, California, San Fran, San Fran...",0,0,0
3,2hop__146174_8682,2,,[{'question': 'Who is the current mayor of the...,,[Linda Norris],0,0,0
4,2hop__61258_79404,2,,[{'question': 'How do you become a justice of ...,,"[Singapore, Republic of Singapore, appointed b...",0,0,0


In [3]:
def _parse_triplets(generation):
    seen_triplets_header = False
    for line in generation.split('\n'):
        if line.startswith('Triplets'):
            seen_triplets_header = True
            continue
        if line.startswith("Answer"):
            return
        if seen_triplets_header and line.strip():
            yield line.strip()

def parse_triplets(generation):
    return [triplet.strip() for triplet in _parse_triplets(generation) if triplet.strip()]

In [4]:
df['generation'] = df['hops'].map(lambda x: x[-1]['qa_result']['generation'])
# df['triplets'] = df['generation'].map(parse_triplets)
df.head()

,id,n_hops,answer,hops,predicted_answer,reference_answers,exact_match,f1,fuzzy_match,generation
0,2hop__592067_529125,2,,[{'question': 'What is the seat of the county ...,,[San Andreas],0,0,0,279200
1,2hop__91929_86706,2,,"[{'question': 'In the US Open, who beat the pe...",,[Novak Djokovic],0,0,0,
2,2hop__507792_754472,2,,[{'question': 'Where did the rapper of Midnigh...,,"[San Francisco, California, San Fran, San Fran...",0,0,0,
3,2hop__146174_8682,2,,[{'question': 'Who is the current mayor of the...,,[Linda Norris],0,0,0,the 79 201201201
4,2hop__61258_79404,2,,[{'question': 'How do you become a justice of ...,,"[Singapore, Republic of Singapore, appointed b...",0,0,0,


In [5]:
success_mask = df['f1'] > 0.7
fail_df = df[~success_mask]
success_df = df[success_mask]
print(len(fail_df), len(success_df), len(df))
print("{:.3f}".format(len(success_df)/len(df)))

299 1 300
0.003


In [6]:
def format_paragraph(paragraph):
    return f"Paragraph {paragraph['idx']} - {paragraph['paragraph_text']}"

def present_hop(hop):
    print('-'*80)
    print(hop['context'])
    print('-'*80)
    print(hop['question'])
    print('-'*80)
    print(hop['qa_result']['generation'])

def present_row(row):
    print(row['id'])
    print()
    print("Success" if row['f1']>0.99 else "Fail")
    print()
    for hop in row['hops']:
        present_hop(hop)
    print()
    print(f"Reference: {row['reference_answers']}")

In [7]:
i = 0
row = df.iloc[i]
present_row(row)

2hop__592067_529125

Fail

--------------------------------------------------------------------------------
# Milton, California
Milton is an unincorporated community in Calaveras County, California. It lies at an elevation of 394 feet (120 m) and is located at . The community is in ZIP code 95684 and area code 209.

# Calaveras County Courthouse
The Calaveras County Courthouse is a historic courthouse building in San Andreas, California. The brick courthouse was built in 1867 and contained the county's courtroom, jail, and sheriff's office; until 1888, executions were also conducted in the building. The county's Hall of Records was built in front of the courthouse in 1893; the two buildings nearly touch and are considered part of the same complex. The building's jail held outlaw Black Bart, a notorious Northern California highwayman, during his 1883 trial. In 1966, the county moved its courthouse to a new building; the old courthouse is now the Calaveras County Museum, which is operat

In [13]:
i = 8
row = fail_df.iloc[i]
present_row(row)

2hop__142965_178655

Fail

--------------------------------------------------------------------------------
# Nine Million Bicycles
"Nine Million Bicycles" is a song written and produced by Mike Batt for the singer Katie Melua's second album, "Piece by Piece". It was released as the album's first single in September 2005 and reached number five on the UK Singles Chart, becoming Melua's first top five hit as a solo artist. It was a finalist for The Record of the Year prize, losing to "You Raise Me Up" by Westlife.

# Ketevan (album)
Ketevan is the sixth studio album by Georgian-British singer Katie Melua, released in the United Kingdom on 16 September 2013 through Dramatico. At birth Melua was given the name Ketevan, but later she adopted the name Katie.
--------------------------------------------------------------------------------
What is the record label of the composer whose fifth studio album was titled Nine Million Bicycles?
-------------------------------------------------------

In [98]:
df['n_triplets'] = df['triplets'].map(len)
df['n_triplets'].describe()

count    2417.000000
mean        2.894084
std         1.077480
min         1.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         9.000000
Name: n_triplets, dtype: float64

In [99]:
poor_triplets_mask = df['n_triplets'] < df['n_hops']
df[poor_triplets_mask].describe()

,n_hops,exact_match,f1,fuzzy_match,n_triplets
count,349.000000,349.000000,349.000000,349.000000,349.000000
mean,3.303725,0.547278,0.638062,0.587393,2.214900
std,0.746265,0.498474,0.440480,0.493010,0.736726
min,2.000000,0.000000,0.000000,0.000000,1.000000
25%,3.000000,0.000000,0.000000,0.000000,2.000000
50%,3.000000,1.000000,1.000000,1.000000,2.000000
75%,4.000000,1.000000,1.000000,1.000000,3.000000
max,4.000000,1.000000,1.000000,1.000000,3.000000


In [100]:
surprise_success_df = df[poor_triplets_mask&success_mask]
len(surprise_success_df)

199

In [102]:
i = random.randint(0, len(surprise_success_df))
row = surprise_success_df.iloc[i]
present_row(row)

3hop1__33845_42197_18397

Success

--------------------------------------------------------------------------------
# Korean War
On 27 June 1950, two days after the KPA invaded and three months before the Chinese entered the war, President Truman dispatched the United States Seventh Fleet to the Taiwan Strait, to prevent hostilities between the Nationalist Republic of China (Taiwan) and the People's Republic of China (PRC). On 4 August 1950, with the PRC invasion of Taiwan aborted, Mao Zedong reported to the Politburo that he would intervene in Korea when the People's Liberation Army's (PLA) Taiwan invasion force was reorganized into the PLA North East Frontier Force. China justified its entry into the war as a response to "American aggression in the guise of the UN".

# Imperialism
Trotsky, and others, believed that the revolution could only succeed in Russia as part of a world revolution. Lenin wrote extensively on the matter and famously declared that Imperialism was the highest sta